In [2]:
import duckdb
from pathlib import Path

duckdb_path = "data/sakila.duckdb"
Path(duckdb_path).unlink(missing_ok=True)

with duckdb.connect(duckdb_path) as conn, open("sql/load_sakila.sql") as ingest_script:
    conn.sql(ingest_script.read())

    description = conn.sql("DESC;").df()
    films = conn.sql("FROM film;").df()

films.head(3)

IOException: IO Error: Cannot open file "c:\users\adelo\de25\sql_lab1_aira_franco\data\sakila.duckdb": The system cannot find the path specified.


In [3]:

# look at the head() of the description dataframe
description.head()

NameError: name 'description' is not defined